# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2808a7e1f0>
├── 'a' --> tensor([[-0.3720,  0.3143,  0.2521],
│                   [ 1.2026, -0.7291, -1.7998]])
└── 'x' --> <FastTreeValue 0x7f2808a7e310>
    └── 'c' --> tensor([[ 0.2339,  0.1445,  1.7674,  0.3716],
                        [-0.8949, -1.0599, -1.3579, -0.3213],
                        [ 0.6218, -0.6165, -0.8225, -0.0922]])

In [4]:
t.a

tensor([[-0.3720,  0.3143,  0.2521],
        [ 1.2026, -0.7291, -1.7998]])

In [5]:
%timeit t.a

59.3 ns ± 0.0845 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2808a7e1f0>
├── 'a' --> tensor([[-2.5856, -1.7035, -2.2327],
│                   [-0.3427,  0.0471,  1.2558]])
└── 'x' --> <FastTreeValue 0x7f2808a7e310>
    └── 'c' --> tensor([[ 0.2339,  0.1445,  1.7674,  0.3716],
                        [-0.8949, -1.0599, -1.3579, -0.3213],
                        [ 0.6218, -0.6165, -0.8225, -0.0922]])

In [7]:
%timeit t.a = new_value

60.9 ns ± 0.0205 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3720,  0.3143,  0.2521],
               [ 1.2026, -0.7291, -1.7998]]),
    x: Batch(
           c: tensor([[ 0.2339,  0.1445,  1.7674,  0.3716],
                      [-0.8949, -1.0599, -1.3579, -0.3213],
                      [ 0.6218, -0.6165, -0.8225, -0.0922]]),
       ),
)

In [10]:
b.a

tensor([[-0.3720,  0.3143,  0.2521],
        [ 1.2026, -0.7291, -1.7998]])

In [11]:
%timeit b.a

57.4 ns ± 0.0402 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3776, -0.7225,  0.2771],
               [ 0.6263,  0.2961,  0.3730]]),
    x: Batch(
           c: tensor([[ 0.2339,  0.1445,  1.7674,  0.3716],
                      [-0.8949, -1.0599, -1.3579, -0.3213],
                      [ 0.6218, -0.6165, -0.8225, -0.0922]]),
       ),
)

In [13]:
%timeit b.a = new_value

495 ns ± 0.242 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

841 ns ± 0.428 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 7.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 582 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 313 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f273bc56880>
├── 'a' --> tensor([[[-0.3720,  0.3143,  0.2521],
│                    [ 1.2026, -0.7291, -1.7998]],
│           
│                   [[-0.3720,  0.3143,  0.2521],
│                    [ 1.2026, -0.7291, -1.7998]],
│           
│                   [[-0.3720,  0.3143,  0.2521],
│                    [ 1.2026, -0.7291, -1.7998]],
│           
│                   [[-0.3720,  0.3143,  0.2521],
│                    [ 1.2026, -0.7291, -1.7998]],
│           
│                   [[-0.3720,  0.3143,  0.2521],
│                    [ 1.2026, -0.7291, -1.7998]],
│           
│                   [[-0.3720,  0.3143,  0.2521],
│                    [ 1.2026, -0.7291, -1.7998]],
│           
│                   [[-0.3720,  0.3143,  0.2521],
│                    [ 1.2026, -0.7291, -1.7998]],
│           
│                   [[-0.3720,  0.3143,  0.2521],
│                    [ 1.2026, -0.7291, -1.7998]]])
└── 'x' --> <FastTreeValue 0x7f273bc56f10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 73.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2808a843d0>
├── 'a' --> tensor([[-0.3720,  0.3143,  0.2521],
│                   [ 1.2026, -0.7291, -1.7998],
│                   [-0.3720,  0.3143,  0.2521],
│                   [ 1.2026, -0.7291, -1.7998],
│                   [-0.3720,  0.3143,  0.2521],
│                   [ 1.2026, -0.7291, -1.7998],
│                   [-0.3720,  0.3143,  0.2521],
│                   [ 1.2026, -0.7291, -1.7998],
│                   [-0.3720,  0.3143,  0.2521],
│                   [ 1.2026, -0.7291, -1.7998],
│                   [-0.3720,  0.3143,  0.2521],
│                   [ 1.2026, -0.7291, -1.7998],
│                   [-0.3720,  0.3143,  0.2521],
│                   [ 1.2026, -0.7291, -1.7998],
│                   [-0.3720,  0.3143,  0.2521],
│                   [ 1.2026, -0.7291, -1.7998]])
└── 'x' --> <FastTreeValue 0x7f273bc56be0>
    └── 'c' --> tensor([[ 0.2339,  0.1445,  1.7674,  0.3716],
                        [-0.8949, -1.0599, -1.3579, -0.3213],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 40.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.9 µs ± 67.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3720,  0.3143,  0.2521],
                [ 1.2026, -0.7291, -1.7998]],
       
               [[-0.3720,  0.3143,  0.2521],
                [ 1.2026, -0.7291, -1.7998]],
       
               [[-0.3720,  0.3143,  0.2521],
                [ 1.2026, -0.7291, -1.7998]],
       
               [[-0.3720,  0.3143,  0.2521],
                [ 1.2026, -0.7291, -1.7998]],
       
               [[-0.3720,  0.3143,  0.2521],
                [ 1.2026, -0.7291, -1.7998]],
       
               [[-0.3720,  0.3143,  0.2521],
                [ 1.2026, -0.7291, -1.7998]],
       
               [[-0.3720,  0.3143,  0.2521],
                [ 1.2026, -0.7291, -1.7998]],
       
               [[-0.3720,  0.3143,  0.2521],
                [ 1.2026, -0.7291, -1.7998]]]),
    x: Batch(
           c: tensor([[[ 0.2339,  0.1445,  1.7674,  0.3716],
                       [-0.8949, -1.0599, -1.3579, -0.3213],
                       [ 0.6218, -0.6165, -0.8225, -0.0922]],
         

In [26]:
%timeit Batch.stack(batches)

80 µs ± 355 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3720,  0.3143,  0.2521],
               [ 1.2026, -0.7291, -1.7998],
               [-0.3720,  0.3143,  0.2521],
               [ 1.2026, -0.7291, -1.7998],
               [-0.3720,  0.3143,  0.2521],
               [ 1.2026, -0.7291, -1.7998],
               [-0.3720,  0.3143,  0.2521],
               [ 1.2026, -0.7291, -1.7998],
               [-0.3720,  0.3143,  0.2521],
               [ 1.2026, -0.7291, -1.7998],
               [-0.3720,  0.3143,  0.2521],
               [ 1.2026, -0.7291, -1.7998],
               [-0.3720,  0.3143,  0.2521],
               [ 1.2026, -0.7291, -1.7998],
               [-0.3720,  0.3143,  0.2521],
               [ 1.2026, -0.7291, -1.7998]]),
    x: Batch(
           c: tensor([[ 0.2339,  0.1445,  1.7674,  0.3716],
                      [-0.8949, -1.0599, -1.3579, -0.3213],
                      [ 0.6218, -0.6165, -0.8225, -0.0922],
                      [ 0.2339,  0.1445,  1.7674,  0.3716],
                      [-0.8949, -

In [28]:
%timeit Batch.cat(batches)

150 µs ± 195 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

325 µs ± 3.03 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
